In [6]:
import pandas as pd
from openpyxl import Workbook
from datetime import datetime
import openpyxl as opx
from openpyxl.styles import Alignment
from openpyxl.styles import PatternFill
import json

In [7]:
import sys
sys.path.append(r'/home/nkrokhmal/umalat')
from app.models_new import *
from app.models_new import Boiling as BoilingModel
from app.models_new import Termizator as TermizatorModel
from openpyxl.utils.cell import coordinate_from_string, column_index_from_string


from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

# sqlite_filepath = '/home/akadaner/Desktop/data.sqlite'
sqlite_filepath = r"/home/nkrokhmal/Desktop/umalat/data.sqlite"
engine = create_engine(f"sqlite:///{sqlite_filepath}")
Session = sessionmaker()
Session.configure(bind=engine)
session = Session()

def cast_sku(obj):
    if isinstance(obj, SKU):
        return obj
    elif isinstance(obj, (str, int)):
        obj = str(obj)
        return session.query(SKU).filter(SKU.id == obj).first()
    else:
        raise Exception('Unknown sku type')
        
def cast_boiling(obj):
    if isinstance(obj, Boiling):
        return obj
    elif isinstance(obj, (str, int)):
        obj = str(obj)
        return session.query(Boiling).filter(Boiling.id == obj).first()
    else:
        raise Exception('Unknown boiling type')

In [4]:
skus = session.query(SKU).all()

In [13]:
df = pd.read_excel(r'request_new.xlsx', index_col=0)
df.columns = range(df.shape[1])

In [14]:
indeces = df.loc['Дата выработки продукции:'].dropna().index
df = df[indeces]

In [15]:
df

In [16]:
tmp = df.loc[['Дата выработки продукции:', 'Заявлено всего, кг:', 'Фактические остатки на складах - Заявлено, кг:', 'Нормативные остатки, кг']].fillna(0)

In [17]:
tmp

In [18]:
tmp.iloc[:, :-1]


In [19]:
tmp = df.dropna(thresh=limit-2)
tmp.dropna(how='all', axis='columns')

In [20]:
sku_list = tmp.iloc[0].dropna()

In [21]:
sku_name = [x for x in list(sku_list) if 'Контрагент' not in x and 'Итог' not in x]

In [35]:
sku_name = list(sku_list[1:-1])

In [37]:
sku_volumes = list(tmp.iloc[-1].dropna()[1:-1])

In [39]:
list(zip(sku_name, sku_volumes))

In [32]:
list(zip(*tmp.values.tolist()))

In [23]:
sku_name

In [31]:
sku_db_names = [x.name for x in skus]

In [32]:
extra_names = [x for x in sku_name if x not in sku_db_names]

In [35]:
extra_names[:-1]

In [43]:
data = ['Кавказский "Умалат" (Перекресток), 45%, кг, в/у',
 'Кавказский "Умалат" (Окей), 45%, кг, в/у',
 'Кавказский "Умалат" (Тандер), 45%, кг , в/у',
 'Кавказский "Умалат" (Метро), 45%, кг, в/у',
 'Кавказский "Умалат", 45%, 0,37 кг, в/у',
 'Кавказский "Глобус", 45%, кг, в/у',
 'Кавказский "Красная птица", 45%, 0,37 кг, в/у',
 'Кавказский "Умалат", 45%, кг, в/у',
 'Сыр Черкесский "Умалат", 45%, 0,28 кг, т/ф',
 'Сыр Черкесский "Умалат", 45%, кг, т/ф, ВЕС',
 'Сыр Черкесский "Умалат" (БИЛЛА), 45%, т/ф, ВЕС',
 'Сыр Черкесский "Умалат" (ДИКСИ), 45%, т/ф, ВЕС',
 'Свели-Квели "Умалат", 30%, 0,37 кг, в/у',
 'Четук "Умалат", 45%, 0,37 кг, в/у',
 'Качорикотта "Unagrande", 45%, 0,37 кг, в/у',
 'Рикотта "Pretto" (зернистая), 30%, 0,37 кг, в/у',
 'Рикотта "Unagrande", 50%, 0,25 кг, пл/с',
 'Рикотта "Unagrande", 50%, 0,5 кг, пл/с',
 'Рикотта "Aventino", 45%, 0,2 кг, п/с',
 'Рикотта "Unagrande Professionale", 45%, 0,5 кг, пл/с',
 'Рикотта с шоколадом "Unagrande", 30%, 0,18 кг, пл/с',
 'Рикотта с шоколадом "Unagrande", 30%, 0,14 кг, пл/с',
 'Рикотта сицилийская "Unagrande", 55%, 0,3 кг, пл/с',
 'Рикотта сицилийская "Unagrande", 55%, ВЕС, пл/с',
 'Рикотта с шоколадом "ВкусВилл", 30%, 0,14 кг, пл/с',
 'Рикотта "Pretto", 45%, 0,5 кг, пл/с',
 'Рикотта "Pretto", 45%, 0,2 кг, пл/с',
 'Рикотта "Фермерская коллекция", 45%, 0,2 кг, пл/с',
 'Рикотта "ВкусВилл", 45%, 0,18 кг, пл/с (6 шт)',
 'Рикотта "Красная птица", 30%,  0,25 кг, пл/с (6 шт)',
 'Рикотта с ванилью "Красная птица", 30%, 0,2 кг, пл/с',
 'Рикотта шоколадно-ореховая "Красная птица", 35%, 0,2 кг, пл/с',
 'Рикотта "SPAR", 30%, 0,2 кг, пл/с (6 шт)',
 'Рикотта с ванилью "Бонджорно", 30%, 0,2 кг, пл/с',
 'Рикотта с шоколадом "Бонджорно", 30%, 0,2 кг, пл/с',
 'Рикотта шоколадно-ореховая "Бонджорно", 35%, 0,2 кг, пл/с',
 'Рикотта "Глобус", 45%, 0,25 кг, пл/с',
 'Сливки Panna Fresca "Unagrande", 38%, 0,5 л, пл/с',
 'Сливки "Красная птица", 38%, 0,25 л, пл/с',
 'Кремчиз "Unagrande", 70%, 0,5 кг, пл/с',
 'Кремчиз "Красная птица", 75%, 0,2 кг, пл/с',
 'Кремчиз "Pretto", 75%, 0,2 кг, пл/с',
 'Кремчиз "ВкусВилл", 70%, 0,18 кг, пл/с (6шт)',
 'Кремчиз "Фермерская коллекция", 75%, 0,2 кг, пл/с',
 'Кремчиз "Unagrande", 70%, 0,18 кг, пл/с',
 'Творожный "Фермерская коллекция", 65%,0,18 кг,пл/с',
 'Творожный "Pretto", 65%, 0,18 кг, пл/с',
 'Робиола "Unagrande", 65%, 0,18 кг, пл/с',
 'Маскарпоне "Unagrande", 80%, 0,25 кг, пл/с',
 'Маскарпоне "Pretto", 80%, 0,25 кг, пл/с',
 'Маскарпоне "Pretto", 80%, 0,5 кг, пл/с',
 'Маскарпоне "Красная птица", 80%,  0,25 кг, пл/с, (6 шт)',
 'Маскарпоне с шоколадом "Красная птица", 50%, 0,2 кг, пл/с',
 'Маскарпоне "ВкусВилл", 80%, 0,25 кг, пл/с (6 шт)',
 'Маскарпоне "Глобус", 80%, 0,25 кг, пл/с',
 'Маскарпоне "Unаgrande", 80%, 0,5 кг, пл/с',
 'Маскарпоне с шоколадом "Бонджорно", 50%, 0,2 кг, пл/с',
 'Маскарпоне с шоколадом "Unagrande", 50%, 0,18 кг, пл/с',
 'Масло сливочное "Умалат", 72,5%  0,5 кг, к/к',
 'Масло сливочное "Умалат", 72,5%, 2 кг, к/к',
 'Масло сладко-сливочное соленое Крестьянское "Unagrande", 72,5%, 0,5 кг, к/к',
 'Масло сладко-сливочное Крестьянское "Unagrande", 72,5%, 0,5 кг, к/к',
 'Масло сладко-сливочное без лактозы Крестьянское "Unagrande", 72,5%, 0,5 кг, к/к',
 'Масло сладко-сливочное традиционное 84%, 2 кг, кор (3 вложения)',
 'Масло сладко-сливочное Традиционное, 82,5%, 2 кг, к/к',
 'Некондиционная продукция',
 'п/ф Рикотта промышленная']

In [44]:
with open('data.json', 'w') as jsonfile:
    json.dump(data, jsonfile, ensure_ascii=False)

In [48]:
with open('ignore_sku.json') as json_file:
    data = json.load(json_file)

In [4]:
skus

In [38]:
def f(sku):
    return {
        'Название SKU': sku.name,
        'Процент': sku.boilings[0].percent,
        'Наличие лактозы': 'Да' if sku.boilings[0].is_lactose else 'Нет',
        'Тип закваски': sku.boilings[0].ferment,
        'Название форм фактора': sku.form_factor.name,
        'Линия': 'Соль' if sku.boilings[0].boiling_type == 'salt' else 'Вода',
        'Имя бренда': sku.brand_name,
        'Вес нетто': sku.weight_netto,
        'Вес форм фактора': sku.boiling_form_factors[0].weight,
        'Выход': sku.output_kg,
        'Срок хранения': sku.shelf_life,
        'Является ли SKU теркой': 'Да' if sku.is_rubber else 'Нет',
        'Упаковщик': sku.packer.name,
        'Тип упаковки': sku.pack_types.name if sku.pack_types else None,
        'Скорость упаковки': sku.packing_speed,
        'Охлаждение 1(для воды)': sku.boiling_form_factors[0].first_cooling_time,
        'Охлаждение 2(для воды)': sku.boiling_form_factors[0].second_cooling_time,
        'Время посолки': sku.boiling_form_factors[0].salting_time,
        'Время налива': sku.boilings[0].pourings.pouring_time,
        'Время отвердевания': sku.boilings[0].pourings.soldification_time,
        'Время нарезки': sku.boilings[0].pourings.cutting_time,
        'Время слива': sku.boilings[0].pourings.pouring_off_time,
        'Дополнительное время': sku.boilings[0].pourings.extra_time        
}

In [39]:
df = pd.DataFrame([f(sku) for sku in skus])

In [40]:
df.to_excel("Все параметры.xlsx")

In [43]:
tmp = pd.read_excel('params.xlsx', index_col=0)

In [44]:
tmp

In [47]:
boiling_technologies_columns = ['Время налива', 'Время отвердевания', 'Время нарезки', 'Время слива', 'Дополнительное время']
btt_data = tmp[boiling_technologies_columns]
# btt_data.drop_duplicates(keep=False)

In [49]:
btt_data.drop_duplicates()

In [50]:
 ['Тип закваски', 'Процент', 'Наличие лактозы', 'Линия'] + boiling_technologies_columns